In this tutorial you'll learn to create your own integration schemes.

# Custom Schemes

In this tutorial we want to estimate $\pi$ with the following equation:

$\pi = 4 \int_0^1 \sqrt{1-t^2}\mathrm{d}t$

In [1]:
from simframe import Frame

In [2]:
sim = Frame()

In [3]:
sim.addfield("pi", 0., description="Approximation of pi")
sim.addintegrationvariable("t", 0.)

In [4]:
import numpy as np

def f(sim, x, Y):
    return 4.*np.sqrt(1-x**2)

In [5]:
sim.pi.differentiator = f

We set the step size to 0.25, i.e., the integral function is only evaluated four times.

In [6]:
def dt(sim):
    return 0.25

In [7]:
sim.t.updater = dt

We do not need to write outputs for this model. We only have to tell the integrator when to stop the simulation, i.e., the upper bound of the integral.

In [8]:
sim.t.snapshots = [1.]

In [9]:
from simframe import Integrator

In [10]:
sim.integrator = Integrator(sim.t)

In [11]:
from simframe import Instruction

In [12]:
from simframe import schemes

In [13]:
sim.integrator.instructions = [Instruction(schemes.expl_1_euler, sim.pi)]

In [14]:
sim.run()

Printing the results

In [15]:
from IPython.display import Markdown as md
def print_table(sim):
    return md("| |$\pi$|rel. error|\n|-|-|-|\n|real|{:10.8f}||\n|approx.|{:10.8f}|{:9.3e}|".format(np.pi,sim.pi,np.abs(np.pi-sim.pi)/np.pi))

In [16]:
print_table(sim)

| |$\pi$|rel. error|
|-|-|-|
|real|3.14159265||
|approx.|3.49570907|1.127e-01|

The relative error is about 11 %. How can we improve it?

One way is to decrease the step size. You can re-run the notebook with a smaller step size and compare the results.

Another possibility is to use higher order integration schemes. We are now going to implement the 2nd-order [Midpoint Method](https://en.wikipedia.org/wiki/Midpoint_method) and the 4th-order [Runge-Kutta method](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods).

All we have to do is write a function that takes the current value of the integration variable **x0**, the current value of the variable to be integrated **Y0**, and the step size **dx**. The function needs to return the change in the variable to be integrated **dY**.

For the midpoint method this looks like this:

In [18]:
def midpoint(x0, Y0, dx, *args, **kwargs):
    x1 = x0 + 0.5*dx
    Y1 = Y0 + 0.5*dx*Y0.derivative(x0, Y0)
    return dx*Y0.derivative(x1, Y1)

We now have to create an integration scheme from this function. This can be done by using `AbstractScheme` provided by `simframe`.

In [19]:
from simframe.integration import AbstractScheme

In [20]:
expl_2_midpoint = AbstractScheme(midpoint)

`expl_2_midpoint` is now our new integration scheme. The naming convention is `<expl/impl>_<order>_<name><_other>`.

We can now set a new instruction set using our new method to the integrator.

In [21]:
sim.integrator.instructions = [Instruction(expl_2_midpoint, sim.pi)]

To re-run the simulation we have to reset to the initial conditions.

In [22]:
sim.t = 0
sim.pi = 0

In [23]:
sim.run()

In [24]:
print_table(sim)

| |$\pi$|rel. error|
|-|-|-|
|real|3.14159265||
|approx.|3.18392922|1.348e-02|

The error is now reduced to 1.3 % only by using a higher order method.

Note: the higher order method needs more operations and is therefore slower. This does not really matter in this case, but might be important for more complex simulations.

The scheme function for the 4th-order explicit Runge-Kutta method looks as follows:

In [25]:
def rk4(x0, Y0, dx, *args, **kwargs):
    k1 = Y0.derivative(x0         , Y0            )
    k2 = Y0.derivative(x0 + 0.5*dx, Y0 + 0.5*dx*k1)
    k3 = Y0.derivative(x0 + 0.5*dx, Y0 + 0.5*dx*k2)
    k4 = Y0.derivative(x0 +     dx, Y0 +     dx*k3)
    return dx*(1/6*k1 + 1/3*k2 + 1/3*k3 + 1/6*k4)

The rest is as above.

In [26]:
expl_4_rungekutta = AbstractScheme(rk4)

In [27]:
sim.integrator.instructions = [Instruction(rk4, sim.pi)]

In [28]:
sim.t = 0
sim.pi = 0

In [29]:
sim.run()

In [30]:
print_table(sim)

| |$\pi$|rel. error|
|-|-|-|
|real|3.14159265||
|approx.|3.12118917|6.495e-03|

With this method the error is reduced down to 0.6 %.

But before you take effort into developing your own integration schemes, take a look at the schemes already provided by `simframe`:

In [31]:
_ = "### Available integration schemes in `simframe.schemes`:\n"
_ += "|Scheme|Description|\n"
_ += "|------|-----------|\n"
for s in schemes.__dir__():
    if s == "update": continue
    if not s.startswith("_"): _ += "|"+s+"|"+schemes.__dict__[s].description+"|\n"
md(_)

### Available integration schemes in `simframe.schemes`:
|Scheme|Description|
|------|-----------|
|expl_1_euler|Explicit 1st-order Euler method|
|expl_2_heun|Explicit 2nd-order Heun's method|
|expl_2_heun_euler_adptv|Explicit adaptive 2nd-order Heun-Euler method|
|expl_2_midpoint|Explicit 2nd-order midpoint method|
|expl_3_heun|Explicit 3rd-order Heun's method|
|expl_4_rungekutta|Explicit 4th-order Runge-Kutta method|
